<a href="https://colab.research.google.com/github/lokeshsai-m/MTP/blob/main/LEDbase_rougeeval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch
torch.cuda.empty_cache()


In [3]:
data = pd.read_csv('/content/dataset1.csv',encoding = 'unicode_escape')
data.drop_duplicates(keep='first')

df = data[['ï»¿abstract', 'conclusion', 'tip', 'no']]

In [4]:
df = df.rename(columns={'no':'id', 'ï»¿abstract':'abstract', 'tip':'summary'})


In [5]:
df['document'] = df['abstract'].apply(lambda x: x.lower())
df['summary'] = df['summary'].apply(lambda x: x.lower())

In [6]:
!pip install transformers==4.17

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


In [7]:
!pip install datasets rouge-score  -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00


In [8]:
!pip install nltk

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
import datasets
from datasets import Dataset
from datasets import load_metric

In [11]:
import transformers
print(transformers.__version__)

4.17.0


In [12]:
from transformers import LEDTokenizer

model_checkpoint ='allenai/led-base-16384'

tokenizer = LEDTokenizer.from_pretrained(model_checkpoint)

pad_on_right = tokenizer.padding_side == "right"

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

In [13]:
max_input_length = 1024
max_target_length = 128

In [14]:
# df = df.sample(frac=1).reset_index(drop=True)
train = df[:75]
valid = df[76:85]
# test = df[47:]
# train.shape, valid.shape,test.shape

In [15]:
def preprocess_function(examples):
    inputs = ['summarize:' + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding='max_length')

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [16]:
import torch
import datasets
from datasets import Dataset

train = Dataset.from_pandas(train)
valid = Dataset.from_pandas(valid)

In [17]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_valid = valid.map(preprocess_function, batched=True)

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

In [18]:
from transformers import TFAutoModelForSeq2SeqLM
from torch import nn

In [19]:
# import transformers
# assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [20]:
from transformers import LEDForConditionalGeneration
model = LEDForConditionalGeneration.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/618M [00:00<?, ?B/s]

In [21]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
batch_size = 4
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [22]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [23]:
import gc
gc.collect()

0

In [24]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("[INFO] training using {}".format(torch.cuda.get_device_name(0)))

[INFO] training using Tesla T4


In [25]:
%env WANDB_DISABLED=True

env: WANDB_DISABLED=True


In [26]:
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-tips",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=True
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [27]:
import tensorflow as tf

In [28]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp half precision backend


In [29]:
metric = load_metric("rouge")

<ipython-input-29-7154407f1f07>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [30]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `LEDForConditionalGeneration.forward` and have been ignored: summary, conclusion, id, abstract, document. If summary, conclusion, id, abstract, document are not expected by `LEDForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 75
  Num Epochs = 20
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 380


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,1.412862,25.903700,12.435200,22.653000,22.405100,19.777800
2,No log,1.350891,27.438600,11.180800,23.322100,23.403600,20.000000
3,No log,1.299656,23.732600,11.677900,21.937600,21.800700,20.000000
4,No log,1.372935,23.206300,9.358300,20.416200,20.008800,20.000000
5,No log,1.438248,30.381900,18.965000,28.647400,28.774400,20.000000
6,No log,1.509392,24.877500,10.225400,22.176500,22.724500,20.000000
7,No log,1.553575,34.396700,20.580900,32.267700,32.824800,20.000000
8,No log,1.593871,32.771800,17.791900,30.941600,31.250900,20.000000
9,No log,1.628619,33.301000,16.843000,28.780500,29.268900,20.000000
10,No log,1.724583,30.406700,16.737500,29.701400,29.601200,19.777800


The following columns in the evaluation set  don't have a corresponding argument in `LEDForConditionalGeneration.forward` and have been ignored: summary, conclusion, id, abstract, document. If summary, conclusion, id, abstract, document are not expected by `LEDForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9
  Batch size = 4
The following columns in the evaluation set  don't have a corresponding argument in `LEDForConditionalGeneration.forward` and have been ignored: summary, conclusion, id, abstract, document. If summary, conclusion, id, abstract, document are not expected by `LEDForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9
  Batch size = 4
The following columns in the evaluation set  don't have a corresponding argument in `LEDForConditionalGeneration.forward` and have been ignored: summary, conclusion, id, abstract, document. If summary

TrainOutput(global_step=380, training_loss=0.711296242161801, metrics={'train_runtime': 290.1125, 'train_samples_per_second': 5.17, 'train_steps_per_second': 1.31, 'total_flos': 1012576813056000.0, 'train_loss': 0.711296242161801, 'epoch': 20.0})

In [31]:
test = df[85:]

In [32]:
print(test.shape)

(12, 5)


In [33]:
eval_dataset = Dataset.from_pandas(test)

In [34]:
eval_dataset = eval_dataset.map(
                preprocess_function,
                batched=True)

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

In [35]:
predict_results = trainer.predict(
            eval_dataset,max_length=128, num_beams=3)

The following columns in the test set  don't have a corresponding argument in `LEDForConditionalGeneration.forward` and have been ignored: summary, conclusion, id, abstract, document. If summary, conclusion, id, abstract, document are not expected by `LEDForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 12
  Batch size = 4


In [36]:
metrics = predict_results.metrics

In [37]:
metrics

{'test_loss': 3.948867082595825,
 'test_rouge1': 22.008,
 'test_rouge2': 5.8394,
 'test_rougeL': 16.595,
 'test_rougeLsum': 17.8788,
 'test_gen_len': 52.3333,
 'test_runtime': 4.9151,
 'test_samples_per_second': 2.441,
 'test_steps_per_second': 0.61}

In [38]:
if args.predict_with_generate:
    predictions = tokenizer.batch_decode(predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    predictions = [pred.strip() for pred in predictions]

In [39]:
predictions

['kodo millets are high in lecithin, which helps against oxidative stress and maintain glucose concentrations in type-2 diabetes. it includes higher amount of antioxidants that help against oxidative stress and maintain glucose concentrations in type-2 diabetes.',
 'honey has hypoglycemic effects, which may be caused by several factors, the most important being lifestyle, but also may be determined by different genes.',
 'foofa beans (vicia fabaâ\xa0l), lentil (lens esculentaâ\xa0), chickpea bread (cicer arietinum-greacumâ\xa0l.), and fenugreek (trigonella foenum-greacumâ\xa0l.) for their antidiabetic activity and mitigation of associated complications, i.e., oxidative stress, liver dysfunction, and lipid metabolism, compared with glibenclamide.',
 'mung beans improve cytoprotective features of the mung bean extract against radical oxygen species, including extract yield, total phenolic content and in vitro antioxidant capacity.',
 'broccoli, lentil, black medick, broccoli, sunflower, 

In [40]:
test['summary']

85    kodo millets have a higher amount of antioxida...
86    honey provides better control of the hyperglyc...
87    faba bean sprouts have the highest level of an...
88    germination increases the content of polypheno...
89    sprouted foods have lower glycemic index and a...
90    microgreens have enhanced nutritional value du...
91    brussel sprouts have favorable effects on oxid...
92    sprouted seeds have enhanced nutritional value...
93    sprouted grainvs have enhanced nutritional val...
94    mustard oil can contribute to the improvement ...
95    spinach improves lipid homeostasis, inflammati...
96    yoga practice including cleansing processes (k...
Name: summary, dtype: object

In [41]:
import csv

# Sample data


# File path
file_path = 'output.csv'

# Writing to CSV
with open(file_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    csvwriter.writerow(['predictions'])
    # Writing each row
    for row in predictions:
        csvwriter.writerow([row])


In [42]:
pip install rouge

In [43]:
import numpy as np
import pandas as pd

In [44]:
data = pd.read_csv('output.csv',encoding = 'unicode_escape')
data.drop_duplicates(keep='first')

,predictions
0,"kodo millets are high in lecithin, which helps..."
1,"honey has hypoglycemic effects, which may be c..."
2,"foofa beans (vicia fabaÃ¢Â l), lentil (lens es..."
3,mung beans improve cytoprotective features of ...
4,"broccoli, lentil, black medick, broccoli, sunf..."
5,microvegetables are often more nutrient-dense ...
6,broccoli sprouts powder may improve oxidative ...
7,"some whole grains are rich sources of energy, ..."
8,chitosan oligosaccharide (cos) and marine prot...
10,"chicken is rich in inorganic nitrate, which re..."


In [45]:
reference = test['summary'].apply(lambda x: x.lower())
predictions1 = data['predictions'].apply(lambda x: x.lower())

In [46]:
reference

85    kodo millets have a higher amount of antioxida...
86    honey provides better control of the hyperglyc...
87    faba bean sprouts have the highest level of an...
88    germination increases the content of polypheno...
89    sprouted foods have lower glycemic index and a...
90    microgreens have enhanced nutritional value du...
91    brussel sprouts have favorable effects on oxid...
92    sprouted seeds have enhanced nutritional value...
93    sprouted grainvs have enhanced nutritional val...
94    mustard oil can contribute to the improvement ...
95    spinach improves lipid homeostasis, inflammati...
96    yoga practice including cleansing processes (k...
Name: summary, dtype: object

In [47]:
for i, prediction in enumerate(predictions1):
            words_list = reference[85+i].split()
            num_words = len(words_list)
            words_list = predictions1[i].split()
            first_words = words_list[:num_words]
            predictions1[i] = ' '.join(first_words)

In [48]:
from rouge import Rouge

def calculate_rouge(reference, predictions):
    rouge = Rouge()

    scores = rouge.get_scores(predictions, reference)
    return scores


for i, prediction in enumerate(predictions1):
            # print(predictions1[0])
            rouge_scores1 = calculate_rouge(reference, predictions1)


print(rouge_scores1)

[{'rouge-1': {'r': 0.5454545454545454, 'p': 0.72, 'f': 0.6206896502675388}, 'rouge-2': {'r': 0.4444444444444444, 'p': 0.6153846153846154, 'f': 0.5161290273881375}, 'rouge-l': {'r': 0.5454545454545454, 'p': 0.72, 'f': 0.6206896502675388}}, {'rouge-1': {'r': 0.17391304347826086, 'p': 0.19047619047619047, 'f': 0.1818181768285125}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.13043478260869565, 'p': 0.14285714285714285, 'f': 0.13636363137396712}}, {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}, {'rouge-1': {'r': 0.1875, 'p': 0.2, 'f': 0.19354838210197725}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.125, 'p': 0.13333333333333333, 'f': 0.12903225306971924}}, {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}, {'rouge-1': {'r': 0.17647058823529413, 'p': 0.16666666666666666, 'f': 0.1714285664326532}

In [49]:
# Initialize variables for average scores
avg_rouge_1_r = 0
avg_rouge_1_p = 0
avg_rouge_1_f = 0

avg_rouge_2_r = 0
avg_rouge_2_p = 0
avg_rouge_2_f = 0

avg_rouge_l_r = 0
avg_rouge_l_p = 0
avg_rouge_l_f = 0

for rouge_dict in rouge_scores1:
    avg_rouge_1_r += rouge_dict['rouge-1']['r']
    avg_rouge_1_p += rouge_dict['rouge-1']['p']
    avg_rouge_1_f += rouge_dict['rouge-1']['f']

    avg_rouge_2_r += rouge_dict['rouge-2']['r']
    avg_rouge_2_p += rouge_dict['rouge-2']['p']
    avg_rouge_2_f += rouge_dict['rouge-2']['f']

    avg_rouge_l_r += rouge_dict['rouge-l']['r']
    avg_rouge_l_p += rouge_dict['rouge-l']['p']
    avg_rouge_l_f += rouge_dict['rouge-l']['f']

# Calculate the average by dividing the sum by the number of elements
num_elements = len(rouge_scores1)
print(num_elements)
avg_rouge_1_r /= num_elements
avg_rouge_1_p /= num_elements
avg_rouge_1_f /= num_elements

avg_rouge_2_r /= num_elements
avg_rouge_2_p /= num_elements
avg_rouge_2_f /= num_elements

avg_rouge_l_r /= num_elements
avg_rouge_l_p /= num_elements
avg_rouge_l_f /= num_elements

# Print the average scores
print("Average ROUGE-1 Recall:", avg_rouge_1_r)
print("Average ROUGE-1 Precision:", avg_rouge_1_p)
print("Average ROUGE-1 F1 Score:", avg_rouge_1_f)

print("Average ROUGE-2 Recall:", avg_rouge_2_r)
print("Average ROUGE-2 Precision:", avg_rouge_2_p)
print("Average ROUGE-2 F1 Score:", avg_rouge_2_f)

print("Average ROUGE-L Recall:", avg_rouge_l_r)
print("Average ROUGE-L Precision:", avg_rouge_l_p)
print("Average ROUGE-L F1 Score:", avg_rouge_l_f)

12
Average ROUGE-1 Recall: 0.17820223994296883
Average ROUGE-1 Precision: 0.22108058608058612
Average ROUGE-1 F1 Score: 0.1953260380856441
Average ROUGE-2 Recall: 0.051742919389978215
Average ROUGE-2 Precision: 0.0700729009552539
Average ROUGE-2 F1 Score: 0.05940696511927449
Average ROUGE-L Recall: 0.14785655698379482
Average ROUGE-L Precision: 0.18534976597476596
Average ROUGE-L F1 Score: 0.16307326160848437


In [50]:
from nltk.translate.bleu_score import sentence_bleu

bleu_scores = [sentence_bleu([ref], pred) for ref, pred in zip(reference, predictions1)]
print(bleu_scores)

[0.6544153527965163, 0.34052850744697893, 0.13069431703303222, 0.27196620119049697, 0.10486427771257309, 0.33202221958981, 0.3969311352842317, 0.4400233483954928, 0.2519264122265381, 0.21004704757293383, 0.21059573510182006, 0.042766130424936585]


In [51]:
# Initialize variables for average scores
avg=0

for val in bleu_scores:
    avg += val


num_elements = len(bleu_scores)

avg/= num_elements

print(avg)

0.28223172373127997
